In [1]:
# To display all outputs in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
#Note: School ID is the unique identifier for a student NOT the name (There are many students of the same name)
#Each school has a Budget
#Aggregate the dataframes to get the mean, sum, and unique counts
#Query DF with specific condition and then aggregate to get the unique counts
#Store in a DF, add proper column names
# Appropriate Display
# Beautify Display

#Construct a 1 column DF with required values as rows
District_Summary = pd.DataFrame(\
pd.concat([school_data_complete.aggregate({'School ID':'nunique', 'Student ID':'nunique', 'math_score':'mean', 'reading_score':'mean'}),\
pd.Series([school_data_complete.groupby(['School ID'] ).agg({'budget':'first'}).sum()[0],\
         100*school_data_complete.query('math_score >= 70')['School ID'].aggregate('count', axis=0)/school_data_complete.shape[0],\
         100*school_data_complete.query('reading_score >= 70')['School ID'].aggregate('count', axis=0)/school_data_complete.shape[0],\
         100*school_data_complete.query('math_score >= 70 & reading_score >= 70')['School ID'].aggregate('count', axis=0)/school_data_complete.shape[0]])]).reset_index(drop=True))

#Take transform
District_Summary = District_Summary.T
#Change Column names
District_Summary.columns = ['Total Schools','Total Students', 'Average Math Score', 'Average Reading Score','Total Budget', '% Passing Math','% Passing Reading','% Overall Passing']

#Change the order of the columns for display
District_Summary = District_Summary[['Total Schools','Total Students', 'Total Budget','Average Math Score', 'Average Reading Score', '% Passing Math','% Passing Reading','% Overall Passing']]
#Formatting for display for the required columns
District_Summary.loc[:, ['Total Schools', 'Total Students', 'Total Budget']] = District_Summary.apply({'Total Schools': lambda x: "{:,.0f}".format(x), 'Total Students': lambda x: "{:,.0f}".format(x), 'Total Budget': lambda x: "${:,.2f}".format(x)})

District_Summary



,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary
* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [125]:
school_summary = school_data_complete.groupby('school_name').aggregate({'type':'first', 'Student ID':'count', 'budget': 'first',\
                                                      'math_score':'mean', 'reading_score':'mean'})

school_summary.columns = ['School Type', 'Total Students', 'Total School Budget', 'Average Math Score','Average Reading Score']
#Add "Per Student Budget" as 'Total School Budget'/'Total Students'

school_summary['Per Student Budget'] = school_summary.apply(lambda x: x['Total School Budget']/x['Total Students'], axis=1)

#% Passing Math, % Passing Reading, % Overall Passing'
#Create DFs with one column for each of  % Passing Math / % Passing Reading / % Passing Reading
#Concatenate these DFs to the school_summary DF based on index
#% Passing Math
math_pass_DF = school_data_complete.loc[school_data_complete.math_score >= 70].groupby('school_name')['Student ID'].count().reset_index()             
math_perc_pass_DF = pd.DataFrame(math_pass_DF.apply(lambda x: 100*x['Student ID']/school_summary.loc[x['school_name'], 'Total Students'], axis=1)) 
math_perc_pass_DF.index = math_pass_DF['school_name']
math_perc_pass_DF.columns = ['% Passing Math']

#% Passing Reading
reading_pass_DF = school_data_complete.loc[school_data_complete.reading_score >= 70].groupby('school_name')['Student ID'].count().reset_index()             
reading_perc_pass_DF = pd.DataFrame(reading_pass_DF.apply(lambda x: 100*x['Student ID']/school_summary.loc[x['school_name'], 'Total Students'], axis=1)) 
reading_perc_pass_DF.index = reading_pass_DF['school_name']
reading_perc_pass_DF.columns = ['% Passing Reading']


#% Overall Passing
overall_pass_DF = school_data_complete.loc[(school_data_complete.math_score >= 70)&(school_data_complete.reading_score >= 70)].groupby('school_name')['Student ID'].count().reset_index()             
overall_perc_pass_DF = pd.DataFrame(overall_pass_DF.apply(lambda x: 100*x['Student ID']/school_summary.loc[x['school_name'], 'Total Students'], axis=1)) 
overall_perc_pass_DF.index = overall_pass_DF['school_name']
overall_perc_pass_DF.columns = ['% Overall Passing']

#Concatenate math_pass_DF, reading_pass_DF and overall_pass_DF to school_summary
school_summary = pd.concat([school_summary, math_perc_pass_DF, reading_perc_pass_DF, overall_perc_pass_DF], axis=1)

#Reorder the column names as per the requirement
school_summary = school_summary[['School Type', 'Total Students', 'Total School Budget','Per Student Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']]       

#Formatting
school_summary[['Total School Budget', 'Per Student Budget']] = school_summary[['Total School Budget', 'Per Student Budget']].applymap(lambda x: "${:,.2f}".format(x))

#Display
school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
